In [1]:
# Imoportar liberias 

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import OPTICS, KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.manifold import TSNE
from sklearn.feature_selection import VarianceThreshold

from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [40]:
# Cargar datos

df_csv = pd.read_csv(r"C:/Users/irwin/Downloads/Sin_GZ.csv")
df_csv
print("Datos cargados con éxito. Dimensiones:", df_csv.shape)
print(df_csv.dtypes)

Datos cargados con éxito. Dimensiones: (38, 47)
POLIZA                  object
SUBRAMO                  int64
INCISO                   int64
SINIESTRO               object
VIGENCIA                object
TIPO DE VEHICULO        object
MARCA VEHICULO          object
DESCRIPCION VEHICULO    object
MODELO                   int64
SERIE VEHICULO          object
ESTADO                  object
POBLACIÓN               object
FECHA DE SINIESTRO       int64
AÑO                      int64
MES                     object
DÍA                     object
RANGO_HORA              object
FECHA DE REPORTE         int64
FECHA CIERRE            object
ESTATUS                 object
NOMBRE CONDUCTOR        object
TIPO PERDIDA            object
CAUSA DETALLADA         object
RESPONSABILIDAD         object
COBERTURA AFECTADA      object
RESERVA                 object
AJUSTES                 object
PAGO                    object
DEDUCIBLE                int64
RECUPERACION             int64
SALVAMENTO            

In [41]:
df_csv['COSTO NETO TOTAL'].unique()

array(['17,280.00', '3,000.00', '0', '22,800.00', '20,419.00',
       '20,934.00', '21,511.00', '16,450.00', '6,813.50', '20,253.23',
       '9,835.39', '12,182.00', '20,967.00', '14,401.99', '39,232.00',
       '17,276.00', '22,257.00', '14,206.68', '91,327.27', '2,000.00',
       '69,828.60', '28,335.24'], dtype=object)

In [42]:
# Limpieza y preprocesamiento de datos

for col in df_csv.columns:
    if df_csv[col].dtype == 'object':
        # Quitar símbolos comunes y convertir a número
        df_csv[col] = (
            df_csv[col]
            .astype(str)    
            .str.replace(',', '', regex=False)
            .str.replace('$', '', regex=False)
            .str.replace('%', '', regex=False)
        )
        df_csv[col] = pd.to_numeric(df_csv[col], errors='ignore')

C:\Users\irwin\AppData\Local\Temp\ipykernel_12264\4245190673.py:13: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_csv[col] = pd.to_numeric(df_csv[col], errors='ignore')
C:\Users\irwin\AppData\Local\Temp\ipykernel_12264\4245190673.py:13: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_csv[col] = pd.to_numeric(df_csv[col], errors='ignore')
C:\Users\irwin\AppData\Local\Temp\ipykernel_12264\4245190673.py:13: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_csv[col] = pd.to_numeric(df_csv[col], errors='ignore')
C:\Users\irwin\AppData\Local\Temp\ipykernel_12264\4245190673.py:13: FutureWarning: errors='ignore' is deprecated and will raise in a 

In [43]:
# Selección de características numéricas para clustering

num_vars = df_csv.select_dtypes(include=[np.number]).columns.tolist()
print("\nVariables numéricas:", num_vars)

X = df_csv[num_vars].copy()
print("\nDimensiones de X:", X.shape)
print("\nValores nulos por columna:\n", X.isna().sum())


Variables numéricas: ['SUBRAMO', 'INCISO', 'MODELO', 'FECHA DE SINIESTRO', 'AÑO', 'FECHA DE REPORTE', 'RESERVA', 'AJUSTES', 'PAGO', 'DEDUCIBLE', 'RECUPERACION', 'SALVAMENTO', 'GASTOS', 'COSTO NETO TOTAL', 'RVA DISPONIBLE', 'CNS_DM', 'CNS_RT', 'CNS_GM', 'CNS_RC', 'CNS_OTROS', 'RVA_DM', 'RVA_RT', 'RVA_RC', 'RVA_GM', 'RVA_OTROS', 'CONTADOR']

Dimensiones de X: (38, 26)

Valores nulos por columna:
 SUBRAMO               0
INCISO                0
MODELO                0
FECHA DE SINIESTRO    0
AÑO                   0
FECHA DE REPORTE      0
RESERVA               0
AJUSTES               0
PAGO                  0
DEDUCIBLE             0
RECUPERACION          0
SALVAMENTO            0
GASTOS                0
COSTO NETO TOTAL      0
RVA DISPONIBLE        0
CNS_DM                0
CNS_RT                0
CNS_GM                0
CNS_RC                0
CNS_OTROS             0
RVA_DM                0
RVA_RT                0
RVA_RC                0
RVA_GM                0
RVA_OTROS             0
C

In [44]:
# Eliminar 

X.dropna(inplace=True)

In [45]:
# Valores faltantes y escalamiento

imputer = SimpleImputer(strategy='mean')  # Rellena NaN con la media
X_imputed = imputer.fit_transform(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

print("\n Datos imputados y escalados. Nueva forma:", X_scaled.shape)


 Datos imputados y escalados. Nueva forma: (38, 26)


In [46]:
# FUNCIONES
def mape(y_true, y_pred):
    y_true = np.array(y_true)
    eps = 1e-8
    return np.mean(np.abs((y_true - y_pred) / (y_true + eps))) * 100

In [47]:
# VARIABLES NUMÉRICAS
num_vars = df_csv.select_dtypes(include=[np.number]).columns.tolist()
print("\nVariables numéricas:", num_vars)



Variables numéricas: ['SUBRAMO', 'INCISO', 'MODELO', 'FECHA DE SINIESTRO', 'AÑO', 'FECHA DE REPORTE', 'RESERVA', 'AJUSTES', 'PAGO', 'DEDUCIBLE', 'RECUPERACION', 'SALVAMENTO', 'GASTOS', 'COSTO NETO TOTAL', 'RVA DISPONIBLE', 'CNS_DM', 'CNS_RT', 'CNS_GM', 'CNS_RC', 'CNS_OTROS', 'RVA_DM', 'RVA_RT', 'RVA_RC', 'RVA_GM', 'RVA_OTROS', 'CONTADOR']


In [49]:
# DEFINIR X Y Y (VARIABLE DEPENDIENTE)

y = df_csv['COSTO NETO TOTAL']  # variable que  se quiere predecir

X = df_csv[num_vars].drop(columns=["COSTO NETO TOTAL"], errors="ignore")




In [50]:
# IMPUTAR NULOS (Muy importante)

imputer = SimpleImputer(strategy="mean")
X_imputed = imputer.fit_transform(X)

In [52]:
# TRAIN / TEST SPLIT

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=38
)

In [54]:
# ESCALAR DATOS

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [55]:
# LASSO CV

lasso = LassoCV(cv=5, random_state=42, n_jobs=-1)
lasso.fit(X_train_scaled, y_train)

y_pred_lasso = lasso.predict(X_test_scaled)

# Realiza regresión LASSO con validación cruzada

c:\Users\irwin\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.643e+06, tolerance: 6.883e+05
  model = cd_fast.enet_coordinate_descent(


In [56]:
# MÉTRICAS LASSO
mse_l = mean_squared_error(y_test, y_pred_lasso)
rmse_l = np.sqrt(mse_l)
mae_l = mean_absolute_error(y_test, y_pred_lasso)
mape_l = mape(y_test, y_pred_lasso)
r2_l = r2_score(y_test, y_pred_lasso)

print("\nLASSOCV (alpha=%.4f)" % lasso.alpha_)
print("MAE: %.4f, RMSE: %.4f, MAPE: %.2f%%, R2: %.4f" %
      (mae_l, rmse_l, mape_l, r2_l))

coef_df = pd.Series(lasso.coef_, index=X.columns).sort_values(key=abs, ascending=False)
print("\nTop coef Lasso:")
print(coef_df.head(10))

# Promedio del cuadrado de los errores
# Promedio del valor absoluto del error
# Porcentaje del valor real de equivocación 


LASSOCV (alpha=15.1469)
MAE: 23.6632, RMSE: 42.8857, MAPE: 54313408924.62%, R2: 1.0000

Top coef Lasso:
CNS_RC            6748.569803
AJUSTES           6344.221126
RESERVA           4564.076095
GASTOS            1078.606623
RVA DISPONIBLE       9.543083
AÑO                  0.000000
SUBRAMO              0.000000
INCISO              -0.000000
MODELO              -0.000000
PAGO                 0.000000
dtype: float64


In [57]:
# RANDOM FOREST

rf = RandomForestRegressor(n_estimators=300, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

# Random Forest es un algoritmo basado en múltiples árboles de decisión.

In [58]:
# MÉTRICAS RF

mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mape_rf = mape(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print("\nRandomForest:")
print("MAE: %.4f, RMSE: %.4f, MAPE: %.2f%%, R2: %.4f" %
      (mae_rf, rmse_rf, mape_rf, r2_rf))

imp = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print("\nTop importancias RF:")
print(imp.head(10))

# Cálculo de métricas de error del modelo Random Forest


RandomForest:
MAE: 7024.0615, RMSE: 18761.9417, MAPE: 1430953958342.02%, R2: 0.5533

Top importancias RF:
CNS_RC                0.660762
RVA DISPONIBLE        0.107087
RVA_RC                0.104363
AJUSTES               0.041357
PAGO                  0.025106
GASTOS                0.021397
INCISO                0.010877
FECHA DE SINIESTRO    0.009071
FECHA DE REPORTE      0.006216
AÑO                   0.005309
dtype: float64


In [ ]:
# GUARDAR RESULTADOS

results = pd.DataFrame({
    'model': ['Lasso', 'RandomForest'],
    'MAE': [mae_l, mae_rf],
    'RMSE': [rmse_l, rmse_rf],
    'MAPE': [mape_l, mape_rf],
    'R2': [r2_l, r2_rf]
})

results.to_csv("supervised_results_summary.csv", index=False)
coef_df.to_csv("lasso_coefficients.csv")
imp.to_csv("rf_feature_importances.csv")

print("\nArchivos guardados correctamente.")




Archivos guardados correctamente.


In [65]:
results = pd.DataFrame({
    'model': ['Lasso', 'RandomForest'],
    'MAE': [mae_l, mae_rf],
    'RMSE': [rmse_l, rmse_rf],
    'MAPE': [mape_l, mape_rf],
    'R2': [r2_l, r2_rf]
})

print("\nResumen de resultados:\n", results)


Resumen de resultados:
           model          MAE          RMSE          MAPE        R2
0         Lasso    23.663245     42.885678  5.431341e+10  0.999998
1  RandomForest  7024.061492  18761.941693  1.430954e+12  0.553250
